# Examples

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Statements" data-toc-modified-id="Import-Statements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Statements</a></span></li><li><span><a href="#Error-in-Cylinder" data-toc-modified-id="Error-in-Cylinder-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Error in Cylinder</a></span><ul class="toc-item"><li><span><a href="#Error-Solver-using-Sympy" data-toc-modified-id="Error-Solver-using-Sympy-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Error Solver using Sympy</a></span></li><li><span><a href="#Error-Solver-using-Written-Module" data-toc-modified-id="Error-Solver-using-Written-Module-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Error Solver using Written Module</a></span></li></ul></li></ul></div>

## Import Statements

In [1]:
import os
import numpy as np
from math import *
from error_solver import ErrorSolver, ErrorSolver2
from error_solver.data import DATA_FOLDER

## Error in Cylinder

The below example determines the tolerances for the cross sectional area $(A)$ and volume $(V)$ of a cylinder given the measurement tolerances for the radius $(r)$ and height $(h)$.

In [2]:
# Variables     Value         Tolerance
r, rtol     =   5,            0.05
h, htol     =   12,           0.05
A, Atol     =   pi * r**2,    None
V, Vtol     =   A * h,        None

# List of equations
equations = ['A = pi * r**2',
             'V = A * h']

# Dictionaries of variables
values = {'h': h,
          'r': r,
          'A': A,
          'V': V}

errors = {'h': rtol,
          'r': htol}

### Error Solver using Sympy

In [3]:
solver = ErrorSolver(equations, values, errors)
solution = solver.solve()

print(solution['summary'])
t1 = %timeit -o solver.solve()

 Variable | Value | Error Tolerance | Percent Error | Unknown 
    h     |  12   |      0.05       |     0.42      |  False  
    r     |   5   |      0.05       |     1.00      |  False  
    A     | 78.54 |      1.571      |     2.00      |  True   
    V     | 942.5 |      22.78      |     2.42      |  True   
7.96 ms ± 933 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Error Solver using Written Module

An alternate error solver is available that does not rely on sympy and is more efficient. However, it requires that functions for the partial derivatives be provided to the initializer. Fortunately, the previous error solver class can be used to write a python module with the requisite functions.

The following methods can be used to develop the module:

In [4]:
path = os.path.join(DATA_FOLDER, 'cylinder_error.py')
names = {} # Substitute variable names if desired
print(solver.module_str(names))
solver.write_module(path, names)

"""
Error Solver Function Module

Equations:
    0 : A - pi*r**2
    1 : -A*h + V
"""

from math import *

# Equation 0
def eq0(A, r, **kwargs):
    return A - pi*r**2

def eq0_A(A, r, **kwargs):
    return 1

def eq0_r(A, r, **kwargs):
    return -2*pi*r

# Equation 1
def eq1(A, V, h, **kwargs):
    return -A*h + V

def eq1_h(A, V, h, **kwargs):
    return -A

def eq1_A(A, V, h, **kwargs):
    return -h

def eq1_V(A, V, h, **kwargs):
    return 1

# Assembled Methods
EQUATIONS = [eq0,
             eq1]

PARTIALS = [{'A' : eq0_A, 'r' : eq0_r},
            {'h' : eq1_h, 'A' : eq1_A, 'V' : eq1_V}]


Since the module was developed using an error solver, the `init_from_module()` method can be used to instantiate the new class.

In [5]:
import error_solver.data.cylinder_error as cylinder_error

solver2 = ErrorSolver2.init_from_module(cylinder_error, values, errors)
solution2 = solver2.solve()

print(solution2['summary'])
t2 = %timeit -o solver2.solve()

 Variable | Value | Error Tolerance | Percent Error | Unknown 
    h     |  12   |      0.05       |     0.42      |  False  
    r     |   5   |      0.05       |     1.00      |  False  
    A     | 78.54 |      1.571      |     2.00      |  True   
    V     | 942.5 |      22.78      |     2.42      |  True   
551 µs ± 55.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
print('For this example, the ErrorSolver2 class is {:.0f}x more efficient than ErrorSolver.'.format(np.mean(t1.timings) / np.mean(t2.timings)))

For this example, the ErrorSolver2 class is 14x more efficient than ErrorSolver.
